In [ ]:
from pprint import pprint

from main import parse_option
from learner import Learner

In [ ]:
parser = parse_option()
method = "padding"
prompt_size = "30"
config = f"--method {method} --prompt_size {prompt_size} --verbose --dataset cifar10 --num_workers 3 --print_freq 50 --visualize --epochs 10 --patience 5"
args = parser(config.split())
pprint(vars(args))

{'arch': 'ViT-B/32',
 'batch_size': 128,
 'dataset': 'cifar10',
 'device': 'cuda',
 'epochs': 10,
 'evaluate': False,
 'filename': 'padding_30_cifar10_clip_ViT-B/32_sgd_lr_40_decay_0_bsz_128_warmup_1000_trial_1',
 'gpu': None,
 'image_dir': './save/images',
 'image_size': 224,
 'learning_rate': 40,
 'method': 'padding',
 'model': 'clip',
 'model_dir': './save/models',
 'model_folder': './save/models/padding_30_cifar10_clip_ViT-B/32_sgd_lr_40_decay_0_bsz_128_warmup_1000_trial_1',
 'momentum': 0.9,
 'num_workers': 3,
 'optim': 'sgd',
 'patience': 5,
 'print_freq': 50,
 'prompt_size': 30,
 'resume': None,
 'root': './data',
 'save_freq': 50,
 'seed': 0,
 'square_size': 8,
 'test_noise': False,
 'text_prompt_template': 'This is a photo of a {}',
 'trial': 1,
 'use_wandb': False,
 'verbose': True,
 'visualize_prompt': True,
 'warmup': 1000,
 'weight_decay': 0}


In [ ]:
learn = Learner(args)

Files already downloaded and verified
Files already downloaded and verified
Building custom CLIP
Loading CLIP (backbone: ViT-B/32)
List of prompts:
['This is a photo of a airplane',
 'This is a photo of a automobile',
 'This is a photo of a bird',
 'This is a photo of a cat',
 'This is a photo of a deer',
 'This is a photo of a dog',
 'This is a photo of a frog',
 'This is a photo of a horse',
 'This is a photo of a ship',
 'This is a photo of a truck']
Visualizing prompt...
Turning off gradients in both the image and the text encoder
("Parameters to be updated: {'prompt_learner.pad_down', "
 "'prompt_learner.pad_left', 'prompt_learner.pad_right', "
 "'prompt_learner.pad_up'}")
Number of prompt parameters:  69840


In [ ]:
train_stats = learn.run()
valid_stats = learn.evaluate("valid")
test_stats = learn.evaluate("test")

  0%|▍                                                                                                                       | 1/313 [00:02<14:51,  2.86s/it]

Epoch: [0][  0/313]	Time  2.858 ( 2.858)	Data  0.943 ( 0.943)	Loss 8.0076e+00 (8.0076e+00)	Acc@1  74.22 ( 74.22)


 16%|███████████████████▍                                                                                                   | 51/313 [00:24<01:53,  2.30it/s]

Epoch: [0][ 50/313]	Time  0.435 ( 0.476)	Data  0.030 ( 0.044)	Loss 6.0171e-01 (9.0305e-01)	Acc@1  83.59 ( 77.19)


 32%|██████████████████████████████████████                                                                                | 101/313 [00:45<01:30,  2.34it/s]

...

Validate: [ 0/79]	Time  1.187 ( 1.187)	Loss 2.7075e-01 (2.7075e-01)	Prompt Acc@1  91.41 ( 91.41)


 65%|█████████████████████████████████████████████████████████████████████████████▍                                          | 51/79 [00:12<00:06,  4.45it/s]

Validate: [50/79]	Time  0.223 ( 0.250)	Loss 2.0532e-01 (2.2646e-01)	Prompt Acc@1  92.97 ( 92.19)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:19<00:00,  4.15it/s]

 * Prompt Acc@1 92.460


In [ ]:
import torch
from torch import nn

In [ ]:
class CheckersPrompt(nn.Module):
    def __init__(self, args):
        super().__init__()

        assert isinstance(args.image_size, int), "image_size must be an integer"
        assert isinstance(args.square_size, int), "square_size must be an integer"
        assert not args.image_size % args.square_size, "image_size must be divisible by square_size"
        self.tile_size = tile_size

        n_tiles = args.image_size ** 2 // 2 + args.image_size % 2
        self.tiles = nn.ParameterList(torch.randn(1, 3, self.tile_size, self.tile_size) for _ in range(n_tiles))

    def forward(self, x):
        tiles_iter = iter(self.tiles)
        for i, row_id in enumerate(1, range(0, x.shape(2), self.tile_size)):
            for j, col_id in enumerate(1, range(0, x.shape(3), self.tile_size)):
                if i % 2: # odd row
                    if j % 2: # odd column
                        x[:, :, row_id : row_id + self.tile_size, col_id : col_id + self.tile_size] += next(tiles_iter)
                else: # even row
                    if not j % 2: # even column
                        x[:, :, row_id : row_id + self.tile_size, col_id : col_id + self.tile_size] += next(tiles_iter)
        return x

In [ ]:
args["image_size"] = 5
args["square_size"] = 1
checkers_prompt = CheckerPrompt(args)
print(checkers_prompt)

TypeError: 'Namespace' object does not support item assignment